In [1]:
import pandas as pd
from IPython.display import display

#plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)

# Bias and fairness audit toolkit
import aequitas
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot


# local modules
from utils import load_dataset, aequitas_preprocessing
%matplotlib inline

Goal of this EDA:
1. Determine if there is bias in the searches


In order to use aequitas we will need to preprocess the dataset by creating two columns:
1. score column: **model prediction**
2. label_value: **ground truth**

And then have categorical columns for aequitas to analyze.

In [2]:
df = load_dataset()
include_columns = ['SubjectRaceCode', 'SubjectEthnicityCode',\
                   'SubjectSexCode', 'InterventionLocationName']
aequitas_df = aequitas_preprocessing(df, include_columns)

In [10]:
g = Group()
xtab, _ = g.get_crosstabs(aequitas_df)

model_id, score_thresholds 1 {'rank_abs': [28341]}


In [28]:
fpr = xtab[['attribute_name', 'attribute_value', 'fpr']].round(2)
print('false positive rates')
display(fpr.head(2))
print(f'shape: {fpr.shape}')

,attribute_name,attribute_value,fpr
0,SubjectRaceCode,A,0.0
1,SubjectRaceCode,B,0.0


shape: (2514, 3)


In [30]:
print(f'rows with at least 1 null: {fpr.isnull().any(axis=1).sum()}')
print(f'rows with all nulls: {fpr.isnull().all(axis=1).sum()}')

rows with at least 1 null: 22
rows with all nulls: 0


We need to understand why some of these values are null. Let's pick **Windsor Locks** location to understand this.

In [14]:
df.head(2)

,VehicleSearchedIndicator,ContrabandIndicator,Department Name,InterventionDateTime,InterventionLocationName,InterventionReasonCode,ReportingOfficerIdentificationID,ResidentIndicator,SearchAuthorizationCode,StatuteReason,SubjectAge,SubjectEthnicityCode,SubjectRaceCode,SubjectSexCode,TownResidentIndicator
0,False,False,New Haven,10/01/2013 12:00:00 AM,NEW HAVEN,V,262,True,N,Stop Sign,31.0,H,W,M,True
1,False,False,State Police,10/01/2013 12:00:00 AM,WILLINGTON,V,1000002715,False,N,Other,29.0,M,W,M,False


In [17]:
windsor_locks = df[df.InterventionLocationName =='Windsor Locks']
# does windsor_locks have any vehicle searched?
windsor_locks[windsor_locks.VehicleSearchedIndicator == True]

,VehicleSearchedIndicator,ContrabandIndicator,Department Name,InterventionDateTime,InterventionLocationName,InterventionReasonCode,ReportingOfficerIdentificationID,ResidentIndicator,SearchAuthorizationCode,StatuteReason,SubjectAge,SubjectEthnicityCode,SubjectRaceCode,SubjectSexCode,TownResidentIndicator
5264,True,False,Windsor Locks,10/04/2013 08:26:00 PM,Windsor Locks,V,212,True,I,Other,21.0,N,W,M,False
5946,True,True,State Police,10/05/2013 11:36:00 AM,Windsor Locks,E,55887736,True,O,Other,27.0,N,W,M,True
31685,True,False,Windsor Locks,10/23/2013 03:58:00 PM,Windsor Locks,V,227,True,I,Suspended License,24.0,N,B,M,False
38836,True,False,Windsor Locks,10/28/2013 08:48:00 PM,Windsor Locks,V,222,True,I,Other,38.0,N,W,M,True
46400,True,False,Windsor Locks,11/03/2013 10:18:00 AM,Windsor Locks,V,206,True,I,Other,23.0,N,W,M,True
52919,True,False,Windsor Locks,11/08/2013 01:18:00 AM,Windsor Locks,V,218,True,O,Speed Related,25.0,N,B,M,False
58996,True,False,Windsor Locks,11/12/2013 01:22:00 PM,Windsor Locks,V,213,True,C,Speed Related,26.0,N,W,M,True
73697,True,False,Windsor Locks,11/21/2013 09:27:00 PM,Windsor Locks,V,206,True,I,Registration,22.0,N,B,M,False
75224,True,False,Windsor Locks,11/22/2013 09:45:00 PM,Windsor Locks,E,62,False,O,Equipment Violation,27.0,N,W,M,False
76847,True,False,Windsor Locks,11/23/2013 08:43:00 PM,Windsor Locks,E,63,True,I,Defective Lights,24.0,N,W,M,False


Now let's visualize our bias!

In [4]:
aqp = Plot()

fpr = aqp.plot_group_metric(xtab, 'fpr')

ValueError: Cannot plot fpr, has NaN values.